In [2]:
import random
from cryptography.fernet import Fernet
import base64

class GarbledANDGate:
    def __init__(self):
        # Generate encryption keys for each wire value
        self.keys = {
            'A0': Fernet.generate_key(),
            'A1': Fernet.generate_key(),
            'B0': Fernet.generate_key(),
            'B1': Fernet.generate_key(),
            'OUT0': Fernet.generate_key(),
            'OUT1': Fernet.generate_key()
        }

        # Create garbled table (double encryption)
        self.garbled_table = self._create_garbled_table()

    def _encrypt(self, key, message):
        fernet = Fernet(key)
        return fernet.encrypt(message.encode())

    def _decrypt(self, key, ciphertext):
        fernet = Fernet(key)
        return fernet.decrypt(ciphertext).decode()

    def _create_garbled_table(self):
        table = []

        # Truth table for AND gate
        truth_table = [
            (0, 0, 0),  # A=0, B=0 → OUT=0
            (0, 1, 0),  # A=0, B=1 → OUT=0
            (1, 0, 0),  # A=1, B=0 → OUT=0
            (1, 1, 1)   # A=1, B=1 → OUT=1
        ]

        for a_val, b_val, out_val in truth_table:
            # Get keys for inputs and output
            key_A = self.keys[f'A{a_val}']
            key_B = self.keys[f'B{b_val}']
            key_OUT = self.keys[f'OUT{out_val}']

            # Encrypt output key with both input keys
            encrypted = self._encrypt(key_A, base64.b64encode(key_OUT).decode())
            double_encrypted = self._encrypt(key_B, base64.b64encode(encrypted).decode())

            table.append(double_encrypted)

        # Shuffle table to hide truth table order
        random.shuffle(table)
        return table

    def get_input_keys(self, party, value):
        """Omar gets A keys, Nancy gets B keys"""
        if party == 'Omar':
            return {value: self.keys[f'A{value}']}
        elif party == 'Nancy':
            return {value: self.keys[f'B{value}']}

    def evaluate(self, key_A, key_B):
        """Evaluate garbled circuit with provided keys"""
        for entry in self.garbled_table:
            try:
                # Try decrypting with B key then A key
                decrypted_B = self._decrypt(key_B, entry)
                decrypted_A = self._decrypt(key_A, base64.b64decode(decrypted_B))
                output_key = base64.b64decode(decrypted_A)

                # Check which output key we got
                if output_key == self.keys['OUT0']:
                    return 0
                elif output_key == self.keys['OUT1']:
                    return 1
            except:
                continue
        return None

# Demonstration
def main():
    print("=== Garbled Circuit AND Gate ===\n")

    # Create circuit
    circuit = GarbledANDGate()

    # Omar's input A=1, Nancy's input B=0
    omar_input = 1
    nancy_input = 0

    print(f"Omar's private input A: {omar_input}")
    print(f"Nancy's private input B: {nancy_input}")
    print(f"Expected A AND B: {omar_input and nancy_input}\n")

    # Parties get their input keys (without knowing other's input)
    omar_keys = circuit.get_input_keys('Omar', omar_input)
    nancy_keys = circuit.get_input_keys('Nancy', nancy_input)

    print("Omar knows his input key but not Nancy's input")
    print("Nancy knows her input key but not Omar's input\n")

    # Evaluate circuit (this could be done by a third party)
    key_A = omar_keys[omar_input]
    key_B = nancy_keys[nancy_input]
    result = circuit.evaluate(key_A, key_B)

    print(f"Computed A AND B: {result}")
    print("✓ Privacy preserved: Neither party learned the other's input!")
    print("✓ Correctness maintained: Got the right AND result")

if __name__ == "__main__":
    main()

=== Garbled Circuit AND Gate ===

Omar's private input A: 1
Nancy's private input B: 0
Expected A AND B: 0

Omar knows his input key but not Nancy's input
Nancy knows her input key but not Omar's input

Computed A AND B: 0
✓ Privacy preserved: Neither party learned the other's input!
✓ Correctness maintained: Got the right AND result
